In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import app_functions as af
import plots as pl

import pandas as pd
import plotly.express as px
import numpy as np

In [230]:
#data_dir_fp = r'Z:\Interés Departamental\Model Portfolio\Hugo\Ranking DIP European Equities\copia 12-04-2023\data\vertical_dowload_files'
#prices_csv_filepath = r'Z:\Interés Departamental\Model Portfolio\Hugo\Ranking DIP European Equities\copia 12-04-2023\data\vertical_dowload_files\PriceClose.csv'
#mask_fp = r'Z:\Interés Departamental\Model Portfolio\Hugo\Ranking DIP European Equities\copia 12-04-2023\data\PriceClose_vertical\monthly_constituents_filter.csv'
data_dir_fp = r'Z:\Interés Departamental\Model Portfolio\Laboratorio de Ideas\Python\prueba'
prices_csv_filepath = r'Z:\Interés Departamental\Model Portfolio\Laboratorio de Ideas\Python\datos Hugo\PriceClose.csv'
mask_fp = r"Z:\Interés Departamental\Model Portfolio\Laboratorio de Ideas\Python\Filtros\stoxx600_month_end_mask.csv"

prices_df = af.read_and_sort_data(prices_csv_filepath)
mask = af.read_and_sort_data(mask_fp)
data_dict = af.filter_data(data_dir_fp,0.8,0.8,600,mask)[0]

Filtering:  PE.csv


In [4]:
pe_df = data_dict['PE']

In [61]:
searching_df = pe_df.loc['2007-01-01':'2010-12-01']

In [64]:
#make window shape relative to df size
all_ranges = np.lib.stride_tricks.sliding_window_view(searching_df.index, window_shape=10)

In [65]:
#set size to be relative to array len (dont watn duplicates)
indices = np.random.randint(0,all_ranges.shape[0], size=30)

In [67]:
samples = all_ranges[indices]

In [140]:
tmp = pd.DataFrame(dict(sample=list(samples)))
tmp['frequency'] = tmp['sample'].apply(pd.infer_freq)

In [150]:
infered_freq = tmp.frequency.value_counts().sort_values().index[0]

In [171]:
def notna_plot(df, relative=False):
    # Create a DataFrame with the number of non-null values in each row of the DataFrame.
    if relative == True:
        data = df.notna().sum(axis=1)/len(df.columns)
    else:
        data = df.notna().sum(axis=1)
    
    fig = px.line(data)
    return fig

In [164]:
n_quantiles = 10

In [162]:
filled_pe_df= pe_df.asfreq(infered_freq)

In [186]:
na_counts = filled_pe_df.notna().sum(axis=1)

In [189]:
changes = (na_counts==0).astype(int).diff()

In [228]:
na_ranges = np.array((changes.iloc[changes.index.get_indexer(changes.loc[changes==1].index)-1].index,changes.loc[changes==-1].index)).T

In [229]:
fig = px.line(na_counts)

for start, end in na_ranges.astype(str):
    print(start, type(start))
    fig.add_vrect(x0=start, x1=end, fillcolor='red', opacity=0.5, line_width=0)

fig.show()

2002-03-31T00:00:00.000000000 <class 'numpy.str_'>
2003-08-31T00:00:00.000000000 <class 'numpy.str_'>
2003-10-31T00:00:00.000000000 <class 'numpy.str_'>
2009-04-30T00:00:00.000000000 <class 'numpy.str_'>
2021-02-28T00:00:00.000000000 <class 'numpy.str_'>
